In [8]:
import asyncio
import json
from playwright.async_api import async_playwright

async def login_and_save_cookies():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        page = await context.new_page()
        await page.goto("https://www.bloomberg.com/")

        print("请在弹出的页面中手动完成登录，然后关闭弹窗或返回这里继续...")

        await asyncio.sleep(180)

        cookies = await context.cookies()
        with open("bloomberg_cookies.json", "w") as f:
            json.dump(cookies, f, indent=2)

        print("Cookies 已保存到 bloomberg_cookies.json")
        await browser.close()

await login_and_save_cookies()


🟢 请在弹出的页面中手动完成登录，然后关闭弹窗或返回这里继续...


In [7]:
import asyncio
from playwright.async_api import async_playwright

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # headless=True 表示无界面运行
        page = await browser.new_page()
        await page.goto("https://example.com")
        print(await page.title())
        await asyncio.sleep(10)  # 等待 10 秒查看网页
        await browser.close()

await run()


Example Domain


CancelledError: 

In [ ]:
import json
import os
import asyncio
from playwright.async_api import async_playwright

async def run(playwright):
    browser = await playwright.chromium.connect_over_cdp('http://localhost:12345/')
    try:
        context = browser.contexts[0]

        
        cookies = await context.cookies()
        print(cookies)

        
        os.makedirs("./dta", exist_ok=True)


        with open("./dta/cookies.json", "w") as f:
            json.dump(cookies, f, indent=2)

        print("Cookies saved to ./dta/cookies.json")

    finally:
        await browser.close()

async def main():
    async with async_playwright() as playwright:
        await run(playwright)

await main()



Error: BrowserType.connect_over_cdp: connect ECONNREFUSED ::1:12345
Call log:
  - <ws preparing> retrieving websocket url from http://localhost:12345/


In [9]:
import json
import os
import time
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright


async def prepare_browser(playwright):
    context = await playwright.chromium.launch_persistent_context(
    user_data_dir="./user_data",
    accept_downloads=True,
    headless=False,
    bypass_csp=True,
    slow_mo=100,
    channel="chromium",
    args=['--disable-blink-features=AutomationControlled'],
    
    user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.122 Safari/537.36",
    viewport={'width': 1280, 'height': 800},
    locale='en-US',
    timezone_id='America/New_York',
    extra_http_headers={
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
    }
)

    if os.path.exists('./dta/cookies.json'):
        with open('./dta/cookies.json', 'r') as f:
            cookies = json.load(f)
        await context.add_cookies(cookies)

    return context


async def save_cookie(context):
    cookies = await context.cookies()
    with open('./dta/cookies.json', 'w') as f:
        json.dump(cookies, f, indent=2)
    await context.close()

async def get_article(context, url):
    page = context.pages[0] if context.pages else await context.new_page()
    await page.goto(url, timeout=5000)
    html = await page.content()
    time.sleep(2) 
    soup = BeautifulSoup(html, 'html.parser')
    article = soup.find('div', class_='body-content')
    return article.get_text(strip=True) if article else "####NO ARTICLE####"


async def run():
    with open('./dta/bloomberg_econ_links.json', 'r') as f:
        data = json.load(f)  

    async with async_playwright() as playwright:
        context = await prepare_browser(playwright)

        os.makedirs('./dta', exist_ok=True)
        with open('./dta/econ_news_articles.json', 'w') as file:
            for idx, url in enumerate(data):
                if idx % 50 == 0 and idx > 0:
                    print('============ renewing browser ============')
                    await save_cookie(context)
                    context = await prepare_browser(playwright)
                
                try:
                    article = await get_article(context, url)
                except Exception as e:
                    print(f"Error at {idx}: {e}")
                    time.sleep(30)
                    context = await prepare_browser(playwright)
                    article = '####ERROR####'
                    
                    continue

                json.dump([article], file)
                file.write('\n')
                print(f"{idx+1}: {article[:50]}...")

        await context.close()

await run()

1: ####NO ARTICLE####...
2: ####NO ARTICLE####...
Error at 2: Page.goto: Target page, context or browser has been closed


In [3]:
!/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=12345 --user-data-dir=/tmp/playwright-profile



DevTools listening on ws://127.0.0.1:12345/devtools/browser/6d3e35cf-2386-44e1-bc4d-21fd9717334d
[56537:3365225:0529/181940.826551:ERROR:chrome/browser/ui/views/frame/browser_view.cc:5932] Attempting to show IPH IPH_TabSearchToolbarButton before browser initialization; IPH will not be shown.
2025-05-29 18:19:41.173 Google Chrome[56537:3365225] +[IMKClient subclass]: chose IMKClient_Legacy
2025-05-29 18:19:41.173 Google Chrome[56537:3365225] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
[56537:3365225:0529/181943.079142:ERROR:chrome/browser/policy/cloud/fm_registration_token_uploader.cc:179] Client is missing for kUser scope
[56537:3365278:0529/181943.121775:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT
Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#-1 is a dynamic-sized tensor).
[565

In [29]:
await run()


In [8]:
import json
import os
import time
import random
import asyncio
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

async def human_like_mouse_movement(page, target_x, target_y, current_x=None, current_y=None):
    """更拟人的鼠标移动路径"""
    if current_x is None or current_y is None:
        current_x = random.randint(200, 800)
        current_y = random.randint(200, 600)
    
    distance = ((target_x - current_x) ** 2 + (target_y - current_y) ** 2) ** 0.5
    
    if distance > 200:
        mid_x = current_x + (target_x - current_x) * 0.6 + random.randint(-50, 50)
        mid_y = current_y + (target_y - current_y) * 0.6 + random.randint(-30, 30)
        
        await page.mouse.move(mid_x, mid_y, steps=random.randint(15, 25))
        await asyncio.sleep(random.uniform(0.1, 0.3))
        await page.mouse.move(target_x, target_y, steps=random.randint(10, 20))
    else:
        steps = max(5, int(distance / 10))
        await page.mouse.move(target_x, target_y, steps=steps)

async def simulate_reading_behavior(page):
    """模拟真实的阅读和浏览行为"""
    behaviors = [
        'scroll_down',
        'scroll_up', 
        'hover_article',
        'quick_scan',
        'pause_thinking'
    ]
    
    selected_behaviors = random.sample(behaviors, random.randint(1, 2))
    
    for behavior in selected_behaviors:
        if behavior == 'scroll_down':
            for _ in range(random.randint(1, 2)):
                await page.mouse.wheel(0, random.randint(150, 300))
                await asyncio.sleep(random.uniform(0.3, 0.8))
                
        elif behavior == 'scroll_up':
            await page.mouse.wheel(0, random.randint(-400, -200))
            await asyncio.sleep(random.uniform(0.4, 1.0))
            
        elif behavior == 'hover_article':
            try:
                articles = await page.query_selector_all('a.story-package-module__story__headline-link')
                if articles:
                    article = random.choice(articles[-10:])
                    box = await article.bounding_box()
                    if box:
                        x = box['x'] + box['width'] / 2
                        y = box['y'] + box['height'] / 2
                        await human_like_mouse_movement(page, x, y)
                        await asyncio.sleep(random.uniform(0.8, 1.5))
            except:
                pass
            
        elif behavior == 'quick_scan':
            for _ in range(random.randint(2, 4)):
                x = random.randint(200, 1000)
                y = random.randint(300, 700)
                await page.mouse.move(x, y, steps=random.randint(5, 12))
                await asyncio.sleep(random.uniform(0.2, 0.6))
            
        elif behavior == 'pause_thinking':
            await asyncio.sleep(random.uniform(1.0, 2.5))
        
        await asyncio.sleep(random.uniform(0.2, 0.5))

async def simulate_human_browsing(page):
    """在爬取文章之间模拟人类浏览行为"""
    behaviors = [
        simulate_reading_behavior,
        self.random_mouse_movements,
        self.check_page_sections,
        self.simulate_headline_reading
    ]
    
    selected_behaviors = random.sample(behaviors, random.randint(1, 3))
    
    for behavior in selected_behaviors:
        await behavior(page)
        await asyncio.sleep(random.uniform(1.0, 3.0))

async def random_mouse_movements(page):
    """随机鼠标移动"""
    movements = random.randint(2, 4)
    for _ in range(movements):
        x = random.randint(100, 1200)
        y = random.randint(200, 800)
        await human_like_mouse_movement(page, x, y)
        await asyncio.sleep(random.uniform(0.2, 0.5))

async def check_page_sections(page):
    """检查页面不同区域"""
    sections = [
        (200, 100),   
        (1000, 300),  
        (150, 500),   
    ]
    
    section = random.choice(sections)
    x, y = section
    await human_like_mouse_movement(page, x, y)
    await asyncio.sleep(random.uniform(0.5, 1.0))

async def simulate_headline_reading(page):
    """模拟标题阅读"""
    try:
        headlines = await page.query_selector_all('h1, h2, h3, .headline')
        if headlines:
            headline = random.choice(headlines[:5])
            box = await headline.bounding_box()
            if box:
                await human_like_mouse_movement(page, box['x'] + 10, box['y'] + box['height']/2)
                await asyncio.sleep(random.uniform(0.8, 1.5))
    except:
        pass

async def prepare_browser(playwright):
    """启动浏览器上下文"""
    context = await playwright.chromium.launch_persistent_context(
        user_data_dir="./user_data",
        accept_downloads=True,
        headless=False,
        bypass_csp=True,
        slow_mo=100,
        channel="chromium",
        args=['--disable-blink-features=AutomationControlled'],
        user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.122 Safari/537.36",
        viewport={'width': 1280, 'height': 800},
        locale='en-US',
        timezone_id='America/New_York',
        extra_http_headers={
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
        }
    )

    if os.path.exists('./dta/cookies.json'):
        with open('./dta/cookies.json', 'r') as f:
            cookies = json.load(f)
        await context.add_cookies(cookies)

    return context

async def save_cookie(context):
    """保存 cookie"""
    cookies = await context.cookies()
    with open('./dta/cookies.json', 'w') as f:
        json.dump(cookies, f, indent=2)

async def get_article(page, url, article_count):
    """获取文章内容，包含人类行为模拟"""
    try:

        await page.goto(url, timeout=5000)
        
        await simulate_reading_behavior(page)
        await asyncio.sleep(random.uniform(2.0, 5.0))
        
        if article_count > 0 and article_count % 5 == 0:
            await simulate_reading_behavior(page)
            await asyncio.sleep(random.uniform(5.0, 10.0))
        
        scroll_count = random.randint(1, 3)
        for _ in range(scroll_count):
            scroll_distance = random.randint(300, 800)
            direction = random.choice([1, -1])
            await page.mouse.wheel(0, scroll_distance * direction)
            await asyncio.sleep(random.uniform(1.0, 2.5))
        

        html = await page.content()
        soup = BeautifulSoup(html, 'html.parser')
        article = soup.find('div', class_='body-content')
        
        await simulate_headline_reading(page)
        await asyncio.sleep(random.uniform(1.0, 3.0))
        
        return article.get_text(strip=True) if article else "####NO ARTICLE####"
    except Exception as e:
        return "####ERROR####"

async def run():
    with open('./dta/bloomberg_econ_links.json', 'r') as f:
        data = json.load(f) 

    async with async_playwright() as playwright:
        context = await prepare_browser(playwright)
        page = await context.new_page()
        
        os.makedirs('./dta', exist_ok=True)
        
        with open('./dta/econ_news_articles.json', 'w') as file:
            for idx, url in enumerate(data):
                if idx > 0 and idx % 10 == 0:
                    print('更换浏览器上下文 (防检测)')
                    await save_cookie(context)
                    await context.close()
                    context = await prepare_browser(playwright)
                    page = await context.new_page()
                    await asyncio.sleep(3)  
                
                if idx > 0 and idx % 3 == 0:
                    print('访问随机页面模拟人类行为')
                    random_pages = [
                        "https://www.bloomberg.com/markets",
                        "https://www.bloomberg.com/technology",
                        "https://www.bloomberg.com/businessweek"
                    ]
                    try:
                        await page.goto(random.choice(random_pages))
                        await simulate_reading_behavior(page)
                        await asyncio.sleep(random.uniform(5.0, 10.0))
                    except Exception as e:
                        print(f"访问随机页面出错: {e}")
                        continue
                
                try:
                    print(f"爬取文章 {idx+1}/{len(data)}: {url[:60]}...")
                    article = await get_article(page, url, idx+1)
                    
                    json.dump([article], file)
                    file.write('\n')
                    
                    wait_time = random.uniform(3.0, 12.0)
                    print(f"随机等待 {wait_time:.1f}秒")
                    await asyncio.sleep(wait_time)
                    
                except Exception as e:
                    print(f"爬取过程中出错: {e}")
                    try:
                        await save_cookie(context)
                        await context.close()
                    except:
                        pass  
                
                    context = await prepare_browser(playwright)
                    page = await context.new_page()
                
                    wait_time = 10
                    print(f"出错后等待 {wait_time}秒...")
                    await asyncio.sleep(wait_time)
                    continue

        await save_cookie(context)
        await context.close()

await run()

爬取文章 1/582: https://www.bloomberg.com/news/articles/2025-05-15/video-is-...


CancelledError: 